# Script to populate MongoDB database 

In [221]:
import pandas as pd
import pymongo
from tqdm import tqdm, tqdm_notebook
from bson.dbref import DBRef

client = pymongo.MongoClient()
db = client.locations
coll = db.states
coll2 = db.places

#### Import US state names, abbreviations, fips ID, and their census division names into Pandas.

In [4]:
states = pd.read_csv('data/states.csv')
state = pd.concat([states.abbreviation, states.name, states.census_division_name, states.fips_state], axis=1)
states.head()

,id,name,abbreviation,country,type,sort,status,occupied,notes,fips_state,assoc_press,standard_federal_region,census_region,census_region_name,census_division,census_division_name,circuit_court
0,1,Alabama,AL,USA,state,10,current,occupied,NaN,1,Ala.,IV,3,South,6,East South Central,11
1,2,Alaska,AK,USA,state,10,current,occupied,NaN,2,Alaska,X,4,West,9,Pacific,9
2,3,Arizona,AZ,USA,state,10,current,occupied,NaN,4,Ariz.,IX,4,West,8,Mountain,9
3,4,Arkansas,AR,USA,state,10,current,occupied,NaN,5,Ark.,VI,3,South,7,West South Central,8
4,5,California,CA,USA,state,10,current,occupied,NaN,6,Calif.,IX,4,West,9,Pacific,9


#### Insert the imported state info into a 'states' MongoDB collection 

In [60]:
for n in tqdm(range(states.shape[0])):
    coll.insert_one(
            {
                "_id": states.iloc[n].abbreviation,
                "fips": int(states.iloc[n].fips_state),
                "name": states.iloc[n]['name'].lower(),
                "census_division": states.iloc[n].census_division_name
            }
    )
    
coll.find_one()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 528.61it/s]


{'_id': 'AL',
 'census_division': 'East South Central',
 'fips': 1,
 'name': 'alabama'}

#### Import places(census term) that reside within the US(not including Peurto Rico et al)

In [3]:
places = pd.read_csv('data/states_places_counties.txt', delimiter="|", encoding="utf8")
state_list = list(states.abbreviation)
places = places[places.STATE.isin(state_list)]
places.head()

,STATE,STATEFP,PLACEFP,PLACENAME,TYPE,FUNCSTAT,COUNTY
0,AL,1,100,Abanda CDP,Census Designated Place,S,Chambers County
1,AL,1,124,Abbeville city,Incorporated Place,A,Henry County
2,AL,1,460,Adamsville city,Incorporated Place,A,Jefferson County
3,AL,1,484,Addison town,Incorporated Place,A,Winston County
4,AL,1,676,Akron town,Incorporated Place,A,Hale County


#### Insert places into a 'places' MongoDB collection

In [62]:
for n in tqdm(range(places.shape[0])):
    if places.STATE.iloc[n] in state_list:
        coll2.insert_one(
                {
                    "name": places.iloc[n].PLACENAME.lower(),
                    "county": places.iloc[n].COUNTY.lower(),
                    "state": DBRef(collection="states", id=str(places.iloc[n].STATE)),
                    "type": places.iloc[n].TYPE.lower(),
                    "func_stat": places.iloc[n].FUNCSTAT.lower(),
                    "fips": int(places.iloc[n].PLACEFP)
                }
        )
coll2.find_one()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41160/41160 [01:00<00:00, 679.71it/s]


{'_id': ObjectId('581d15cf9e8de2482c1bc3f3'),
 'county': 'chambers county',
 'fips': 100,
 'func_stat': 's',
 'name': 'abanda cdp',
 'state': DBRef('states', 'AL'),
 'type': 'census designated place'}

#### Import counties - their fips ID, their state's fips ID

In [4]:
counties = pd.read_csv("data/county_fips.txt")
counties = counties[counties.state.isin(state_list)]
counties['county_name'] = counties['county_name'].str.lower()
county_names = list(counties.county_name)
counties.head()

,state,state_fips,county_fips,county_name,func
0,AL,1,1,autauga county,H1
1,AL,1,3,baldwin county,H1
2,AL,1,5,barbour county,H1
3,AL,1,7,bibb county,H1
4,AL,1,9,blount county,H1


#### Add county fips IDs to the places collection

In [64]:
for c in tqdm(counties.iterrows()):
    coll2.update_many({"county": c[1]['county_name']}, {"$set": {"county_fips": c[1]['county_fips']}})
coll2.find_one()

3142it [01:18, 40.48it/s]


{'_id': ObjectId('581d15cf9e8de2482c1bc3f3'),
 'county': 'chambers county',
 'county_fips': 71,
 'fips': 100,
 'func_stat': 's',
 'name': 'abanda cdp',
 'state': DBRef('states', 'AL'),
 'type': 'census designated place'}

#### Import county crime rates

In [65]:
county_crime = pd.read_csv("data/county_crime.txt",delimiter="\t")
county_crime.head()

,STUDYNO,EDITION,PART,IDNO,FIPS_ST,FIPS_CTY,CPOPARST,AG_ARRST,JURFLAG,COVIND,...,OFAGFAM,DUI,LIQUOR,DRUNK,DISORDR,VAGRANT,ALLOTHR,SUSPICN,CURFEW,RUNAWAY
0,9999,1,1,1,1,1,57217,3,1,0.0000,...,0,84,21,68,12,3,260,0,0,0
1,9999,1,1,2,1,3,198843,14,0,10.4765,...,5,427,87,239,36,6,2126,0,0,0
2,9999,1,1,3,1,5,27026,5,0,32.4354,...,0,35,9,28,6,1,110,0,0,0
3,9999,1,1,4,1,7,22491,4,1,27.9504,...,0,19,0,1,8,0,90,0,0,0
4,9999,1,1,5,1,9,58037,6,0,7.3561,...,2,52,51,23,4,0,499,0,0,0


#### Assign county crime rates to the places collection

In [66]:
for county in tqdm(county_crime.iterrows()):
    pop_served = county[1]['CPOPARST'] # 0 in certain circumstances - forrests, bridges, roads
    if pop_served == 0: # Can't calculate rate if population served is 0
        continue
    st_f = county[1]['FIPS_ST']
    ct_f = county[1]['FIPS_CTY']
    st = coll.find_one({"fips": st_f}, {"_id": 1})
    grnd_tot = county[1]['GRNDTOT'] # Some may have a 0 val(tread carefully)
    coll2.update_many({"county_fips": ct_f},{"$set": {"crime_rate": grnd_tot/pop_served}})
    
coll2.find_one()

3177it [01:57, 27.15it/s]


{'_id': ObjectId('581d15cf9e8de2482c1bc3f3'),
 'county': 'chambers county',
 'county_fips': 71,
 'crime_rate': 0.06453368965431916,
 'fips': 100,
 'func_stat': 's',
 'name': 'abanda cdp',
 'state': DBRef('states', 'AL'),
 'type': 'census designated place'}

#### Acquiring the state crime rates based on the county crime values and population groupings

In [67]:
state_crime = county_crime.groupby('FIPS_ST')['GRNDTOT'].sum()
state_crime_pop = county_crime.groupby('FIPS_ST')['CPOPARST'].sum()
print(state_crime.head(2))
print(state_crime_pop.head(2))

FIPS_ST
1    76900
2    26840
Name: GRNDTOT, dtype: int64
FIPS_ST
1    4849377
2     736732
Name: CPOPARST, dtype: int64


#### Importing the state crime rates into the state collection

In [68]:
for state in tqdm(state_crime.iteritems()):
    coll.update_one({"fips": int(state[0])}, {"$set": {"crime_rate": state[1]/state_crime_pop[state[0]]}})
coll.find_one()

51it [00:00, 2682.35it/s]


{'_id': 'AL',
 'census_division': 'East South Central',
 'crime_rate': 0.0158577070827861,
 'fips': 1,
 'name': 'alabama'}

#### Builds the zipcode files and the associated HPI rates

In [69]:
zip_county = pd.read_csv('data/zip_county_cross.csv')

zip_hpi = pd.read_csv('data/housing_HPI_zip.csv')
zip_hpi = zip_hpi[zip_hpi['Year']==2015]

county_series = zip_county['COUNTY'].unique()
zips_in_county = []
for c in tqdm(county_series):
    zip_list = list(zip_county[zip_county['COUNTY'] == c]['ZIP'])
    zips_in_county.append((c, zip_list))

# Removes the counties which do not have any HPI-containing zipcodes in them
zips_in_county = [z for z in zips_in_county if len(z[1]) != 0]

# Remove entries from hpi_zips_list which have the rates set as '.'
zip_hpi = zip_hpi[zip_hpi['Annual Change (%)'] != '.']

county_rate_dict = {'county': [], 'avg_rate': [], 'state': []}

# Removes zip codes not found in hpi_zips_list from zips_in_county
hpi_zips_list = list(zip_hpi['Five-Digit ZIP Code'])
for zc in tqdm(range(len(zips_in_county))):
    zc1 = [zipp for zipp in zips_in_county[zc][1] if zipp in hpi_zips_list]
    zips_in_county[zc] = (zips_in_county[zc][0],zc1)

# Remove counties with no zip codes from zips_in_county
zips_in_county = [zi for zi in zips_in_county if len(zi[1]) != 0]


for county_zips in tqdm(zips_in_county):
    county_rate_dict['county'].append(int(str(county_zips[0])[-3:]))
    county_rate_dict['state'].append(int(str(county_zips[0])[0:-3]))
    zip_vals = [float(zip_hpi[zip_hpi['Five-Digit ZIP Code'] == x]['Annual Change (%)'].iloc[0]) for x in county_zips[1]]
    county_rate_dict['avg_rate'].append(sum(zip_vals)/len(zip_vals))
county_rate_dict['state_abbr'] = []

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2763/2763 [00:14<00:00, 191.15it/s]


In [70]:
county_rate_dict.keys()

dict_keys(['county', 'state_abbr', 'avg_rate', 'state'])

In [71]:
to_del = []
for xx in range(len(county_rate_dict['state'])):
    if county_rate_dict['state'][xx] not in states['fips_state'].values:
        to_del.append(xx)
county_rate_dict['county'] = [z[1] for z in enumerate(county_rate_dict['county']) if z[0] not in to_del]
county_rate_dict['state'] = [z[1] for z in enumerate(county_rate_dict['state']) if z[0] not in to_del]
county_rate_dict['avg_rate'] = [z[1] for z in enumerate(county_rate_dict['avg_rate']) if z[0] not in to_del]

for sti in county_rate_dict['state']:
    stabbr = states['abbreviation']
    abbr = stabbr[states['fips_state']==sti].iloc[0]
    county_rate_dict['state_abbr'].append(abbr)

#### Import the HPI county rates to their respective places in the MongoDB places collection

In [72]:
for county_rate in tqdm(enumerate(county_rate_dict['county'])):
    coll2.update({'state': DBRef('states', county_rate_dict['state_abbr'][county_rate[0]]),
                  'county_fips': county_rate[1]},
                 {"$set": {'hpi_rate': county_rate_dict['avg_rate'][county_rate[0]]}})

0it [00:00, ?it/s]C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
2762it [00:53, 51.92it/s]


In [73]:
coll2.find_one()

{'_id': ObjectId('581d15cf9e8de2482c1bc3f3'),
 'county': 'chambers county',
 'county_fips': 71,
 'crime_rate': 0.06453368965431916,
 'fips': 100,
 'func_stat': 's',
 'hpi_rate': 7.052,
 'name': 'abanda cdp',
 'state': DBRef('states', 'AL'),
 'type': 'census designated place'}

#### Import state HPI values and calculate the rates

In [74]:
hpi_states = pd.read_csv('data/hpi_states.csv',names=['state', 'year', 'q', 'hpi'])
hpi_states = hpi_states[(hpi_states['year']==2014) | (hpi_states['year']==2015)]
hpi_states['hpi'] = hpi_states['hpi'].astype(float)
g = hpi_states.groupby(['state', 'year'])
g_hpi_states = g['hpi'].mean()
state_hpi_rates = []
for st in tqdm(g_hpi_states.index.levels[0]):
    state_hpi_rates.append((st,(g_hpi_states[st][2015]-g_hpi_states[st][2014])/g_hpi_states[st][2014]))

state_hpi_rates[0:10]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 1413.97it/s]


[('AK', 0.033933518005540231),
 ('AL', 0.0310071987368759),
 ('AR', 0.025619307650915794),
 ('AZ', 0.065579932181231343),
 ('CA', 0.073942752364212896),
 ('CO', 0.11204488986190218),
 ('CT', 0.018820869003392093),
 ('DC', 0.069974057103079301),
 ('DE', 0.030671091401172534),
 ('FL', 0.097044610531876754)]

#### Insert the state HPI rates into their respective states in the states collection

In [75]:
for st in tqdm(state_hpi_rates):
    coll.update_one({'_id': st[0]}, {"$set": {"hpi": st[1]}})

coll.find_one()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 2766.87it/s]


{'_id': 'AL',
 'census_division': 'East South Central',
 'crime_rate': 0.0158577070827861,
 'fips': 1,
 'hpi': 0.0310071987368759,
 'name': 'alabama'}

#### Imports median housing cost/month rates

In [76]:
med_housing = pd.read_csv('data/acs_median_housing.csv', encoding="latin-1")
med_housing.drop(0, inplace=True)

med_housing.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01
1,1600000US0100100,0100100,"Abanda CDP, Alabama",-,**
2,1600000US0100124,0100124,"Abbeville city, Alabama",542,66
3,1600000US0100460,0100460,"Adamsville city, Alabama",777,130
4,1600000US0100484,0100484,"Addison town, Alabama",413,83
5,1600000US0100676,0100676,"Akron town, Alabama",605,204


#### Provides a list of states and their fips IDs

In [77]:
state_fips_tup_list = [(st[1]['fips_state'],st[1]['abbreviation']) for st in states.iterrows()]
state_fips_dict = dict(state_fips_tup_list)

#### Adds the median montly housing costs to the places collection

In [78]:
dindex = 0
for mh in tqdm(med_housing.iterrows()):
    try:
        mon_cost = int(mh[1]['HD01_VD01'])
    except ValueError:
        continue
        
    geoid = mh[1]['GEO.id2']
    place_id = int(geoid[2:])
    state_id = int(geoid[0:2])
    if state_id in state_fips_dict.keys():
        try:
            st_abbr = state_fips_dict[state_id]
        except:
            print(state_id, place_id, geoid)
            raise
        coll2.update_many({'state': DBRef('states', st_abbr), 'fips': place_id}, {'$set': {'median_housing_costs': mh[1]['HD01_VD01']}})
        
coll2.find_one()

29296it [12:56, 37.75it/s]


{'_id': ObjectId('581d15cf9e8de2482c1bc3f3'),
 'county': 'chambers county',
 'county_fips': 71,
 'crime_rate': 0.06453368965431916,
 'fips': 100,
 'func_stat': 's',
 'hpi_rate': 7.052,
 'name': 'abanda cdp',
 'state': DBRef('states', 'AL'),
 'type': 'census designated place'}

#### Import state median housing costs

In [90]:
med_housing_st = pd.read_csv('data/acs_median_housing_states.csv')
med_housing_st.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01
0,0400000US01,1,Alabama,745,3
1,0400000US02,2,Alaska,1248,12
2,0400000US04,4,Arizona,967,4
3,0400000US05,5,Arkansas,678,3
4,0400000US06,6,California,1427,2


#### Insert the median housing cost into the state collection

In [96]:
for housing_cost in tqdm(med_housing_st.iterrows()):
    coll.update_one({'fips': housing_cost[1]['GEO.id2']}, {'$set': {'median_housing_costs': housing_cost[1]['HD01_VD01']}})

coll.find_one()

51it [00:00, 1544.65it/s]


{'_id': 'AL',
 'census_division': 'East South Central',
 'crime_rate': 0.0158577070827861,
 'fips': 1,
 'hpi': 0.0310071987368759,
 'median_housing_costs': 745,
 'name': 'alabama'}

#### Import places education data and metadata

In [74]:
ed_places = pd.read_csv('data/places_education.csv', encoding='latin-1')
ed_places['HD02_VD01'] = pd.to_numeric(ed_places['HD02_VD01'], errors='coerce')
ed_places['GEO.id2'] = ed_places['GEO.id2'].astype(str)
print(ed_places.head())

ed_places_meta = pd.read_csv('data/places_education_meta.csv', names = [0,1])
print(ed_places_meta)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


             GEO.id GEO.id2         GEO.display-label  HD01_VD01  HD02_VD01  \
0  1600000US0100100  100100       Abanda CDP, Alabama         43       51.0   
1  1600000US0100124  100124   Abbeville city, Alabama       1947      110.0   
2  1600000US0100460  100460  Adamsville city, Alabama       3243      249.0   
3  1600000US0100484  100484     Addison town, Alabama        683      203.0   
4  1600000US0100676  100676       Akron town, Alabama        201       54.0   

   HD01_VD02  HD02_VD02  HD01_VD03  HD02_VD03  HD01_VD04    ...      \
0          0         11          0         11          0    ...       
1         41         28          0         11          0    ...       
2         27         25          0         11          0    ...       
3          7          8          0         11          0    ...       
4          3          4          0         11          0    ...       

   HD01_VD21  HD02_VD21  HD01_VD22  HD02_VD22  HD01_VD23  HD02_VD23  \
0          0         11    

In [75]:
# Returns the education data label based on the index value
def n2v(n):
    return ed_places_meta[0].iloc[n]

#### Codify the different education levels

In [76]:
# From no education to a 12th grade education classified as '0'
ed_places[0] = 0
for n in [5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33]:
    ed_places[0] = ed_places[0] + ed_places[n2v(n)]

# High School or high school equivalent and less than one year of college classified as '1' 
ed_places[1] = 0
for n in [35, 37, 39]:
    ed_places[1] = ed_places[1] + ed_places[n2v(n)]

# College educated(bachelor's, associates's, or one+ years of college) will be classified as '2'
ed_places[2] = 0
for n in [41, 43, 45]:
    ed_places[2] = ed_places[2] + ed_places[n2v(n)]

# Professional school and Masters educations will be classified as '3'
ed_places[3] = 0
for n in [47, 49]:
    ed_places[3] = ed_places[3] + ed_places[n2v(n)]
    
# Doctorate degree will be classified as '4'
ed_places[4] = 0
for n in [51]:
    ed_places[4] = ed_places[4] + ed_places[n2v(n)]

In [93]:
for n in range(5):
    ed_places[n] = ed_places[n]/ed_places[n2v(3)]

#### Insert the education rates of the places into an embedded document within the places of the places collection

In [210]:
errlog = []
for pl in tqdm(ed_places.iterrows()):
    if ed_places[n2v(3)] == 0:
        continue
    geoid = pl[1]['GEO.id2']
    st_id = int(geoid[0:-5])
    if st_id in state_fips_dict.keys():
        st_abbr = state_fips_dict[st_id]
        pl_id = int(geoid[-5:])
        coll2.update_many({'state': DBRef('states', st_abbr), 'fips': pl_id}, {'$set': {'ed_level': {'1': pl[1][1],
                                                                                                    '2': pl[1][2],
                                                                                                    '3': pl[1][3],
                                                                                                    '4': pl[1][4],
                                                                                                    '0': pl[1][0]}}})

29296it [12:51, 37.97it/s]


In [214]:
coll2.find_one()

{'_id': ObjectId('581d15cf9e8de2482c1bc3f3'),
 'county': 'chambers county',
 'county_fips': 71,
 'crime_rate': 0.06453368965431916,
 'ed_level': {'0': 0.7209302325581395,
  '1': 0.27906976744186046,
  '2': 0.0,
  '3': 0.0,
  '4': 0.0},
 'fips': 100,
 'func_stat': 's',
 'hpi_rate': 7.052,
 'name': 'abanda cdp',
 'state': DBRef('states', 'AL'),
 'type': 'census designated place'}

#### Build state education rates out of imported places amounts

In [73]:
# Adds a state id column derived from the GEO.id2 for grouping purposes
st_id = ed_places['GEO.id2'].apply(lambda x: x[:-5])
ed_places['st_id'] = st_id
ed_places['st_id'].head()

NameError: name 'ed_places' is not defined

In [223]:
# Groups the values by state and sums up their values
ed_st = ed_places.groupby('st_id').sum()
ed_st.head()

,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,...,HD02_VD23,HD01_VD24,HD02_VD24,HD01_VD25,HD02_VD25,0,1,2,3,4
st_id,,,,,,,,,,,,,,,,,,,,,
1,2068193,108285.0,25404,16116,138,7211,234,7204,903,7560,...,31911,37766,15966,25668,13546,116.235729,240.629223,188.712132,29.532942,2.889973
10,256969,17578.0,3145,2397,29,988,58,942,114,1055,...,5885,5066,2538,5169,2434,9.195610,29.197507,29.484819,7.892781,1.229283
11,442721,87.0,4996,596,39,37,71,75,250,118,...,1836,37053,1419,17109,967,0.110903,0.219820,0.368422,0.262210,0.038645
12,10274116,397005.0,148390,63884,2313,17163,1770,17071,7279,19957,...,127875,209386,66279,113454,49977,135.872025,350.076917,349.992879,70.708044,9.350135
13,3013654,142189.0,40336,24344,398,9472,626,9465,2081,10476,...,47690,69822,23919,43130,19786,135.060682,253.193386,195.203988,36.046461,4.495482


#### Codify state educational levels, similar to those of the places educational levels above

In [225]:
# From no education to a 12th grade education classified as '0'
ed_st[0] = 0
for n in [5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33]:
    ed_st[0] = ed_st[0] + ed_st[n2v(n)]

# High School or high school equivalent and less than one year of college classified as '1' 
ed_st[1] = 0
for n in [35, 37, 39]:
    ed_st[1] = ed_st[1] + ed_st[n2v(n)]

# College educated(bachelor's, associates's, or one+ years of college) will be classified as '2'
ed_st[2] = 0
for n in [41, 43, 45]:
    ed_st[2] = ed_st[2] + ed_st[n2v(n)]

# Professional school and Masters educations will be classified as '3'
ed_st[3] = 0
for n in [47, 49]:
    ed_st[3] = ed_st[3] + ed_st[n2v(n)]
    
# Doctorate degree will be classified as '4'
ed_st[4] = 0
for n in [51]:
    ed_st[4] = ed_st[4] + ed_st[n2v(n)]

In [226]:
for n in range(5):
    ed_st[n] = ed_st[n]/ed_st[n2v(3)]
ed_st.head()

,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,...,HD02_VD23,HD01_VD24,HD02_VD24,HD01_VD25,HD02_VD25,0,1,2,3,4
st_id,,,,,,,,,,,,,,,,,,,,,
1,2068193,108285.0,25404,16116,138,7211,234,7204,903,7560,...,31911,37766,15966,25668,13546,0.148762,0.344379,0.405843,0.088605,0.012411
10,256969,17578.0,3145,2397,29,988,58,942,114,1055,...,5885,5066,2538,5169,2434,0.123746,0.355895,0.389950,0.110293,0.020115
11,442721,87.0,4996,596,39,37,71,75,250,118,...,1836,37053,1419,17109,967,0.110903,0.219820,0.368422,0.262210,0.038645
12,10274116,397005.0,148390,63884,2313,17163,1770,17071,7279,19957,...,127875,209386,66279,113454,49977,0.135086,0.354333,0.412609,0.086930,0.011043
13,3013654,142189.0,40336,24344,398,9472,626,9465,2081,10476,...,47690,69822,23919,43130,19786,0.153681,0.317791,0.408510,0.105707,0.014312


#### Insert the education levels into the states of the states collection

In [255]:
for st in tqdm(ed_st.iterrows()):
    if int(st[0]) in state_fips_dict.keys():
        coll.update_one({'fips': int(st[0])}, {'$set': {'ed_level': {'1': st[1][1],
                                                                                                    '2': st[1][2],
                                                                                                    '3': st[1][3],
                                                                                                    '4': st[1][4],
                                                                                                    '0': st[1][0]}}})
coll.find_one()

51it [00:00, 1787.58it/s]


{'_id': 'AL',
 'census_division': 'East South Central',
 'crime_rate': 0.0158577070827861,
 'ed_level': {'0': 0.14876174515627894,
  '1': 0.34437936885000575,
  '2': 0.4058431684083642,
  '3': 0.08860488358678324,
  '4': 0.012410833998567832},
 'fips': 1,
 'hpi': 0.0310071987368759,
 'median_housing_costs': 745,
 'name': 'alabama'}

#### Importing cost of living index and CBSA crosswalk

In [260]:
cbsa = pd.read_csv('data/metro_micro_county_crosswalk.csv', encoding = 'latin-1')
cbsa.head()

,CBSA Code,Metropolitan Division Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Metropolitan Division Title,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,Central/Outlying County
0,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Brown County,South Dakota,46,13,Central
1,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Edmunds County,South Dakota,46,45,Outlying
2,10140,NaN,NaN,"Aberdeen, WA",Micropolitan Statistical Area,NaN,NaN,Grays Harbor County,Washington,53,27,Central
3,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,NaN,NaN,Callahan County,Texas,48,59,Outlying
4,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,NaN,NaN,Jones County,Texas,48,253,Outlying


In [263]:
coli = pd.read_csv('data/coli.csv')
coli.head()

,CITY CODE,STATE,METRO/MICRO,URBAN AREA AND STATE,INDEX
0,01-11500-100,Alabama,Anniston-Oxford-Jacksonville AL Metro,Anniston-Calhoun County AL,86.3
1,01-12220-125,Alabama,Auburn-Opelika AL Metro,Auburn-Opelika AL,96.0
2,01-13820-200,Alabama,Birmingham-Hoover AL Metro,Birmingham AL,89.5
3,01-19460-235,Alabama,Decatur AL Metro,Decatur-Hartselle AL,86.5
4,01-20020-250,Alabama,Dothan AL Metro,Dothan AL,85.3


In [282]:
# Decomposes the CITY CODE into the state and cbsa fips identifiers
coli['st'] = pd.to_numeric(coli['CITY CODE'].apply(lambda x: x[:2]))
coli['cbsa_f'] = pd.to_numeric(coli['CITY CODE'].apply(lambda x: x[3:8]))

coli.head()

,CITY CODE,STATE,METRO/MICRO,URBAN AREA AND STATE,INDEX,st,cbsa_f
0,01-11500-100,Alabama,Anniston-Oxford-Jacksonville AL Metro,Anniston-Calhoun County AL,86.3,1,11500
1,01-12220-125,Alabama,Auburn-Opelika AL Metro,Auburn-Opelika AL,96.0,1,12220
2,01-13820-200,Alabama,Birmingham-Hoover AL Metro,Birmingham AL,89.5,1,13820
3,01-19460-235,Alabama,Decatur AL Metro,Decatur-Hartselle AL,86.5,1,19460
4,01-20020-250,Alabama,Dothan AL Metro,Dothan AL,85.3,1,20020


#### Insert the price indexes into places collection

In [300]:
for co in tqdm_notebook(coli.iterrows()):
    st = co[1]['st']
    cbsa_f = co[1]['cbsa_f']
    index = co[1]['INDEX']
    for county in tqdm(cbsa[cbsa['CBSA Code'] == cbsa_f].iterrows(), leave=False):
        state_f = county[1]['FIPS State Code']
        try:
            st_abbr = state_fips_dict[state_f]
        except:
            pass
        county_f = county[1]['FIPS County Code']
        coll2.update_many({'county_fips': county_f, 'state': DBRef('states', st_abbr)}, {'$set': {'price_index': index}})

coll2.find_one()

{'_id': ObjectId('581d15cf9e8de2482c1bc3f3'),
 'county': 'chambers county',
 'county_fips': 71,
 'crime_rate': 0.06453368965431916,
 'ed_level': {'0': 0.7209302325581395,
  '1': 0.27906976744186046,
  '2': 0.0,
  '3': 0.0,
  '4': 0.0},
 'fips': 100,
 'func_stat': 's',
 'hpi_rate': 7.052,
 'name': 'abanda cdp',
 'state': DBRef('states', 'AL'),
 'type': 'census designated place'}

In [303]:
coll2.find({'price_index': {'$exists': 1}}).count()

10912

#### Insert price indexes into the states collection

In [317]:
ind_state = coli.groupby('st')['INDEX'].mean()
ind_state.head()
for col in ind_state.iteritems():
    coll.update_one({'fips': int(col[0])}, {'$set': {'price_index': col[1]}}) #int64 needed to be converted to a native Python int to work with BSON

coll.find_one()

{'_id': 'AL',
 'census_division': 'East South Central',
 'crime_rate': 0.0158577070827861,
 'ed_level': {'0': 0.14876174515627894,
  '1': 0.34437936885000575,
  '2': 0.4058431684083642,
  '3': 0.08860488358678324,
  '4': 0.012410833998567832},
 'fips': 1,
 'hpi': 0.0310071987368759,
 'median_housing_costs': 745,
 'name': 'alabama',
 'price_index': 89.85555555555555}

#### Import 2016 Election Data

In [3]:
import json
with open('data/election_map.json') as dat:
    parsed_json = json.loads(dat.read())

In [12]:
parsed_json[1].keys()

dict_keys(['candidates', 'key_race', 'counties', 'state_slug', 'race_id', 'report', 'nyt_rating', 'reporting_display', 'votes', 'state_id', 'electoral_votes', 'poll_display', 'precincts_total', 'office', 'race_type', 'uncontested', 'result', 'poll_time', 'has_incumbent', 'result_source', 'election_type', 'precincts_reporting', 'guid', 'path', 'race_name', 'election_date', 'race_slug'])

#### Insert County Election Data into Places as republican and democrate rates

In [4]:
for elec_state in tqdm_notebook(parsed_json):
    s_abbr = elec_state['state_id']
    #print(s_abbr)
    for elec_county in tqdm_notebook(elec_state['counties'], leave= False):
        #print(elec_county['name'])
        if 'est_votes_remaining' in elec_county:
            if elec_county['est_votes_remaining'] == 0:
                cnty_f = int(elec_county['fips'][2:])
                votes = elec_county['votes']
                rep = elec_county['results']['trumpd']/votes
                dem = elec_county['results']['clintonh']/votes
                #print(rep, dem)
                coll2.update_many({'state': DBRef('states', s_abbr), 'county_fips': cnty_f}, {'$set': {'republican': rep, 'democrat': dem }})

In [10]:
for elec_state in tqdm_notebook(parsed_json):
    s_abbr = elec_state['state_id']
    for candidate in elec_state['candidates']:
        if candidate['candidate_key'] == 'trumpd':
            coll.update_one({'_id': s_abbr}, {'$set': {'republican': candidate['percent']}})
        elif candidate['candidate_key'] == 'clintonh':
            coll.update_one({'_id': s_abbr}, {'$set': {'democrat': candidate['percent']}})

In [18]:
coll.find_one({'_id': 'MO'})

{'_id': 'MO',
 'census_division': 'West North Central',
 'crime_rate': 0.044650618635267,
 'democrat': 38,
 'ed_level': {'0': 0.1182740374440974,
  '1': 0.3636961054387611,
  '2': 0.40775455781120146,
  '3': 0.09764087167633963,
  '4': 0.012634427629600433},
 'fips': 29,
 'hpi': 0.040215618771516456,
 'median_housing_costs': 817,
 'name': 'missouri',
 'price_index': 90.91666666666667,
 'republican': 57.1}